# 15 STAC specification

The **SpatioTemporal Asset Catalog (STAC)** is an emerging open standard for geospatial data that aims to increase the interoperability of geospatial data, particularly satellite imagery. 
[Many major data archives](https://stacspec.org/en/about/datasets/) now follow the STAC specification.

In this lesson we'll be working with the [Microsoft's Planetary Computer (MPC)](https://planetarycomputer.microsoft.com) STAC API. 
In this lesson we will learn about the main components of a STAC catalog and how to search for data using the MPC's STAC API. 

## MPC Catalog 
First, load the necessary packages:

In [1]:
# import libraries

import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rioxarray as rioxr
from pystac_client import Client # to access STAC catalogs

import planetary_computer # to sign items from the MPC STAC Catalog

from IPython.display import Image # this is to nicely display images

### Access
We use the `Client` function from the `pystac_client` package to access the catalog:

In [3]:
# Access MPC catalog
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

The `modifier` parameter is needed to access the data in the MPC catalog.

### Catalog Exploration


In [4]:
# Explore catalog metadata
print('Title: ', catalog.title)
print('Description: ', catalog.description)

Title:  Microsoft Planetary Computer STAC API
Description:  Searchable spatiotemporal metadata describing Earth science datasets hosted by the Microsoft Planetary Computer


We can access its collections by using the `get_collections()` method:

In [5]:
catalog.get_collections()

<generator object Client.get_collections at 0x7f91bba9b560>

The output of `get_collections()` is a **generator**:
    
    - a special kind of lazy object in Python and you can loop over it as in a list
    - a list of items that dont exist in memory until you iterate over it or convert them to list 
    - kind of disposable, can't be reused
    - allows for more efficient momory management
    once the generator is iterated over completely, it cannot be reused unless recreated

In [6]:
collections = list(catalog.get_collections()) # Turn generator into a list

print('Number of collections: ', len(collections))

Number of collections:  124


In [7]:
print('Collection IDS (first 10): ')
for i in range(10):
    print('-', collections[i].id) # indexing with the i for iteration

Collection IDS (first 10): 
- daymet-annual-pr
- daymet-daily-hi
- 3dep-seamless
- 3dep-lidar-dsm
- fia
- sentinel-1-rtc
- gridmet
- daymet-annual-na
- daymet-monthly-na
- daymet-annual-hi


## Collection

The NAIP catalogs ID is `naip`. We can select a single collection for exploration using `get_child()` method for the catlog and the colelction ID as a parameter:

In [8]:
naip_collection = catalog.get_child('naip')
naip_collection

<CollectionClient id=naip>

## Catalog search

We can narrow down the seach within the catalog by specifying:
- time range
- area of interest
- collection name

The simplest way to define the area of interest to look for data in the catalog is to define:

- a geojson-type dictionary with the coordiantes of the boudning box we want to loop for data over

Alternatively:
- a list`[xmin, ymin, xmax, ymax]` with the coordinates

In [9]:
# Define NCEAS bounding box (as a GeoJSON)
bbox = {
    "type": "Polygon",
    "coordinates":[
        [
            [-119.70608227128903, 34.426300194372274], # this is the same as the last one to close the polygon
            [-119.70608227128903, 34.42041139020533],
            [-119.6967885126002, 34.42041139020533],
            [-119.6967885126002, 34.426300194372274],
            [-119.70608227128903, 34.426300194372274] # same as first
        ]
    ],
}

# Define temporal range of interest
time_range = "2018-01-01/2023-01-01"

In [11]:
# Catalog search
search = catalog.search(
    collections =['naip'],
    intersects = bbox,
    datetime = time_range)

search

To get the items found in the search we use the `item_collection` method:

In [12]:
# Retrieve items
items = search.item_collection()
len(items)

3

## Item
Let's get the first item in the search:

In [14]:
# get first item in search
item= items[0]
item

<Item id=ca_m_3411935_sw_11_060_20220513>

Remember that the STAC item is the core object in the STAC catalog. The item does not contain the data itself, but rather it has metadata and assets that contain links to the actual data. Some of the metadata.

In [16]:
# Print item ID and properties
print('ID', item.id)
item.properties

ID ca_m_3411935_sw_11_060_20220513


{'gsd': 0.6,
 'datetime': '2022-05-13T16:00:00Z',
 'naip:year': '2022',
 'proj:bbox': [246930.0, 3806808.0, 253260.0, 3814296.0],
 'proj:epsg': 26911,
 'providers': [{'url': 'https://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip-imagery/',
   'name': 'USDA Farm Service Agency',
   'roles': ['producer', 'licensor']}],
 'naip:state': 'ca',
 'proj:shape': [12480, 10550],
 'proj:centroid': {'lat': 34.40624, 'lon': -119.71877},
 'proj:transform': [0.6, 0.0, 246930.0, 0.0, -0.6, 3814296.0, 0.0, 0.0, 1.0]}

In [17]:
item.assets

{'image': <Asset href=https://naipeuwest.blob.core.windows.net/naip/v002/ca/2022/ca_060cm_2022/34119/m_3411935_sw_11_060_20220513.tif?st=2024-11-24T18%3A08%3A19Z&se=2024-11-25T18%3A53%3A19Z&sp=rl&sv=2024-05-04&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2024-11-25T17%3A41%3A28Z&ske=2024-12-02T17%3A41%3A28Z&sks=b&skv=2024-05-04&sig=hckDXUFAsEohsK1AFbR/dOLf0it996H3VfSFtXpmGBk%3D>,
 'thumbnail': <Asset href=https://naipeuwest.blob.core.windows.net/naip/v002/ca/2022/ca_060cm_2022/34119/m_3411935_sw_11_060_20220513.200.jpg?st=2024-11-24T18%3A08%3A19Z&se=2024-11-25T18%3A53%3A19Z&sp=rl&sv=2024-05-04&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2024-11-25T17%3A41%3A28Z&ske=2024-12-02T17%3A41%3A28Z&sks=b&skv=2024-05-04&sig=hckDXUFAsEohsK1AFbR/dOLf0it996H3VfSFtXpmGBk%3D>,
 'tilejson': <Asset href=https://planetarycomputer.microsoft.com/api/data/v1/item/tilejson.json?collection=naip&item=ca_m_3411

In [18]:
for key in item.assets.keys():
    print(key, '--', item.assets[key].title)

image -- RGBIR COG tile
thumbnail -- Thumbnail
tilejson -- TileJSON with default rendering
rendered_preview -- Rendered preview


In [21]:
# Plot rendered preview
Image(url=item.assets['rendered_preview'].href, width = 500) # accessing rendered image from dictionary

## Load data

The raster data in our current `item` is the image asset
We access this data via its URL.
This time we open it using `rioxr.open_rasterio()`:

In [22]:
sb = rioxr.open_rasterio(item.assets['image'].href)

In [23]:
sb

<xarray.DataArray (band: 4, y: 12480, x: 10550)> Size: 527MB
[526656000 values with dtype=uint8]
Coordinates:
  * band         (band) int64 32B 1 2 3 4
  * x            (x) float64 84kB 2.469e+05 2.469e+05 ... 2.533e+05 2.533e+05
  * y            (y) float64 100kB 3.814e+06 3.814e+06 ... 3.807e+06 3.807e+06
    spatial_ref  int64 8B 0
Attributes:
    TIFFTAG_IMAGEDESCRIPTION:  OrthoVista
    TIFFTAG_SOFTWARE:          Trimble Germany GmbH
    TIFFTAG_XRESOLUTION:       1
    TIFFTAG_YRESOLUTION:       1
    TIFFTAG_RESOLUTIONUNIT:    1 (unitless)
    AREA_OR_POINT:             Area
    scale_factor:              1.0
    add_offset:                0.0